In [ ]:
import lateral_signaling as lsig

import os
from pathlib import Path
from glob import glob
import json

import numpy as np
import pandas as pd

import skimage
import skimage.io as io
import skimage.filters as filt
import skimage.measure as msr

import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
import bebi103

import bokeh.io
bokeh.io.output_notebook()

---

In [ ]:
## WARNING
## Before setting to True, make sure to back up any work that could be overwritten
save_data = False

We are interested in images where all the information exists within a circular region of interest (ROI). In order to extract this information from an image, we first sample points along the edge of the circular well. This returns (x, y) coordinates in units of pixels. Then, we compute the least-squares estimate for the center and radius of the circle [1]. We can then use this information to re-center and rescale images onto the same coordinates. This does lead to slight variations in the XY position and/or magnification, so we adjust for this later in the pipeline.

Next, we want to quantify the expression along one axis of the well, for which we will draw a line profile (LP). This is a rectangular ROI defined by a source point, destination point, and width in pixels. In subsequent analysis, the intensity of every pixel in this ROI is projected down onto the line segment from source to destination, giving us a 1D summary of intensity along an arbitrary direction of the image.

[1]: Randy Bullock, "Least-Squares Circle Fit." [Link](https://dtcenter.org/sites/default/files/community-code/met/docs/write-ups/circle_fit.pdf). email:bullock@ucar.edu

---

# Kinematic wave

These experiments were done by imaging the same well with the same microscope settings, so we only need to draw a single ROI.

__Set up directories__

In [ ]:
# For reading
image_dir         = lsig.data_dir.joinpath("imaging/kinematic_wave")
well_diam_path    = lsig.data_dir.joinpath("imaging/well_diameter.json")

# For writing
circle_data_path  = lsig.analysis_dir.joinpath("kinematic_wave/roi_circle.json")
lp_data_path      = lsig.analysis_dir.joinpath("kinematic_wave/line_profile.json")

# Read well diameter from file
with well_diam_path.open("r") as f:
    well_diameter_mm = json.load(f)["kinematic_wave"]

print(f"Well diameter: {well_diameter_mm:.2f} mm")

__Load images and image metadata__

In [ ]:
# Get image filenames
files = sorted(glob(os.path.join(image_dir, "*.tif*")))
files = [os.path.realpath(f) for f in files]

In [ ]:
# Select blue and green fluorescence images (BFP and GFP)
files_B = [f for f in files if "BFP" in f]
files_G = [f for f in files if "GFP" in f]

In [ ]:
# Load images and convert to Numpy arrays
load_B = lambda f: io.imread(f).astype(np.uint8)[:, :, 2]
ims_B = io.ImageCollection(files_B, load_func=load_B).concatenate()

load_G = lambda f: io.imread(f).astype(np.uint8)[:, :, 1]
ims_G = io.ImageCollection(files_G, load_func=load_G).concatenate()

In [ ]:
# Get images as Numpy array
ims = list(ims_B) + list(ims_G)

# Save shape of each image
imshape_B = ims_B[0].shape
imshape_G = ims_G[0].shape

## Draw circular ROI

In [ ]:
# # Uncomment to preview the image
# im = lsig.rescale_img(ims_B[0])
# plt.imshow(im)

In [ ]:
# Draw an ROI onto the image by clicking along the border of the well
circle_verts_roicds = bebi103.image.draw_rois(im, flip=False, min_intensity=0.0, max_intensity=0.9)

In [ ]:
# Once satisfied, convert the result to a Pandas DataFrame
circle_verts_df = bebi103.image.roicds_to_df(circle_verts_roicds)
circle_verts_df

## Compute the center and radius of the ROI and save

In [ ]:
# Calculate least-squares estimate of center and radius
center, radius = lsig.verts_to_circle(circle_verts)

# Store data
circle_data = dict(
    x_center=center[0], 
    y_center=center[1], 
    radius=radius,
    well_diameter_mm=well_diameter_mm,
)

# Dump to JSON file
if save_data:
    with open(circle_data_path, "w") as f:
        json.dump(circle_data, f, indent=4, )

## Draw a line profile (LP)

In [ ]:
# # Select images to use when drawing source and destination points of line profile
# roi_ims = [lsig.rescale_img(ims[5]), lsig.rescale_img(ims[9])]

In [ ]:
# # Decide the source point by clicking on the first image
# lp_vert1 = bebi103.image.record_clicks(roi_ims[0], flip=False, min_intensity=0.1, max_intensity=0.5)

In [ ]:
# # Draw destination point
# lp_vert2 = bebi103.image.record_clicks(roi_ims[1], flip=False, min_intensity=0.1, max_intensity=0.4)

In [ ]:
# Set width of line profile in pixels
lp_width = 500.

# Package into dictionary
lp_data = dict(
    x_src = lp_vert1.data["x"], 
    y_src = lp_vert1.data["y"], 
    x_dst = lp_vert2.data["x"], 
    y_dst = lp_vert2.data["y"], 
    width = lp_width
)

# Save as JSON file
if save_data:
    with open(lp_data_path, "w") as f:
        json.dump(lp_data, f, indent=4, )

<hr>

# Signaling gradient

The images in this experiment are stitched (collated from multiple images), so there are slight variations in the well size, centering, and general orientation. Assuming no rotational effects, we draw an ROI for each image and use the ROI data to transform the image data to the same coordinates for analysis in real distance units. 

In [ ]:
# For reading
image_dir         = lsig.data_dir.joinpath("imaging/signaling_gradient")
well_diam_path    = lsig.data_dir.joinpath("imaging/well_diameter.json")

# For writing
circle_data_path  = lsig.analysis_dir.joinpath("signaling_gradient/roi_circle_data.csv")
lp_data_path      = lsig.analysis_dir.joinpath("signaling_gradient/line_profile.json")

# Read well diameter from file
with well_diam_path.open("r") as f:
    well_diameter_mm = json.load(f)["signaling_gradient"]

print(f"Well diameter: {well_diameter_mm:.2f} mm")

__Load images__

In [ ]:
# Get image filenames
files = sorted(glob(os.path.join(image_dir, "*.tif*")))
files = [os.path.realpath(f) for f in files]

In [ ]:
# Select blue and green fluorescence images (BFP and GFP)
files_B = [f for f in files if "BFP" in f]
files_G = [f for f in files if "GFP" in f]

In [ ]:
# Load images and convert to Numpy arrays
load_B = lambda f: io.imread(f).astype(np.uint8)[:, :, 2]
ims_B = io.ImageCollection(files_B, load_func=load_B).concatenate()

load_G = lambda f: io.imread(f).astype(np.uint8)[:, :, 1]
ims_G = io.ImageCollection(files_G, load_func=load_G).concatenate()

In [ ]:
# Get images as Numpy array
ims = list(ims_B) + list(ims_G)

# Save shape of each image
imshape_B = ims_B[0].shape
imshape_G = ims_G[0].shape

## Draw circular ROI(s)

In [ ]:
# Initialize a dictionary to hold the ROI parameters for each image
roi_circle_dict = {}

The below cells should be run repeatedly, once for each image, to populate `roi_circle_dict`.

In [ ]:
# # Select one image
# im_idx = 0

In [ ]:
# # Draw an ROI by clicking along the border of the well
# im = ims[im_idx]
# circle_verts_roicds = bebi103.image.draw_rois(im, flip=False,)

In [ ]:
# # Find center and radius
# circle_df = bebi103.image.roicds_to_df(circle_verts_roicds)
# circle_verts = circle_df.values[:, 1:]
# center, radius = lsig.verts_to_circle(circle_verts)

# # Store in dictionary
# roi_circle_dict[im_names[im_idx]] = [*center, radius]

## Save circle parameters

In [ ]:
roi_circle_df = pd.DataFrame(roi_circle_dict).T.reset_index()
roi_circle_df.columns = ["im_name", "x_center", "y_center", "radius"]
roi_circle_df

if save_data:
    circle_verts_df.to_csv(circle_data_path, index=False)

## Draw a line profile (LP)

In [ ]:
# # Select images to use when drawing source and destination points of line profile
# roi_ims = [lsig.rescale_img(ims[5]), lsig.rescale_img(ims[9])]

In [ ]:
# # Decide the source point by clicking on the first image
# lp_vert1 = bebi103.image.record_clicks(roi_ims[0], flip=False, min_intensity=0.1, max_intensity=0.5)

In [ ]:
# # Draw destination point
# lp_vert2 = bebi103.image.record_clicks(roi_ims[1], flip=False, min_intensity=0.1, max_intensity=0.4)

In [ ]:
# Set width of line profile in pixels
lp_width = 500.

# Package into dictionary
lp_data = dict(
    x_src = lp_vert1.data["x"], 
    y_src = lp_vert1.data["y"], 
    x_dst = lp_vert2.data["x"], 
    y_dst = lp_vert2.data["y"], 
    width = lp_width,
)

# Save as JSON file
if save_data:
    with open(lp_data_path, "w") as f:
        json.dump(lp_data, f, indent=4, )